# BERT feature extraction for each given dataset

## Librares and utilities

In [1]:
!pip install tape_proteins
#!pip install awscli --ignore-installed six

In [2]:
import torch
from tape import ProteinBertModel, TAPETokenizer
from tape import UniRepModel
import numpy as np
model = ProteinBertModel.from_pretrained('bert-base')
tokenizer = TAPETokenizer(vocab='iupac')

In [3]:
def clip_long_seq(seq, max_len):
  if(len(seq) > max_len):
    seq = seq[0:max_len]
  return seq
def pad_sequence(seq,max_len):
  while(len(seq) < max_len):
    seq = seq + "<pad>"
  return seq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Feature vector for Dataset 1

In [4]:
entries = []
with  open('Database.txt') as fp:
    contents = fp.read()
    for entry in contents.split('-'):
      entry = entry.replace('\n','')
      entries.append(entry)
entries.pop(0)


''

In [5]:
n = 30
strings_30 = []
for entry in entries:
  split_strings = [entry[index : index + n] for index in range(0, len(entry), n)]
  '''for string in split_strings:
    if(len(string)<n):
      split_strings = split_strings[:-1]
      string = pad_sequence(string,n)
      split_strings.append(string)'''

  strings_30.append(split_strings)


In [6]:
list_len = []
for string in strings_30:
  list_len.append(len(string))

In [7]:
np.sum(list_len)

971603

In [ ]:
#test_entries = entries[1:10]
ctr = 0
file_write = open("output_primary.txt","w")
for string in strings_30:
  for s in string:
    s = clip_long_seq(s, 1024)
    token_ids = torch.tensor([tokenizer.encode(s)])
    output = model(token_ids)
    sq = torch.squeeze(output[0],dim=0)
    avg_output = torch.mean(sq,0)
    avg_output_corr = torch.clip(avg_output, min=-1,max=1)
    #print(torch.mean(avg_output_corr))
    file_write.write(str(ctr)+"\n")
    file_write.write(str(avg_output_corr) + "\n")
    ctr+=1
    #pooled_output = output[1]
    #file_write.write(str(avg_output_corr) + "\n")

C:\Users\migue\AppData\Local\Temp/ipykernel_16696/428392496.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  token_ids = torch.tensor([tokenizer.encode(s)])


In [ ]:
#test_entries = entries[1:10]
ctr = 0
file_write = open("/content/drive/MyDrive/output_primary.txt","w")
for entry in entries:
  entry = clip_long_seq(entry, 1024)
  token_ids = torch.tensor([tokenizer.encode(entry)])
  output = model(token_ids)
  sq = torch.squeeze(output[0],dim=0)
  avg_output = torch.mean(sq,0)
  avg_output_corr = torch.clip(avg_output, min=-1,max=1)
  #print(torch.mean(avg_output_corr))
  ctr+=1
  file_write.write(str(avg_output_corr) + "\n")
  #pooled_output = output[1]
  #file_write.write(str(avg_output_corr) + "\n")
  

## Feature vector for secondary structure predictions

In [ ]:
prediction_file = '/content/drive/MyDrive/prediction.txt'
file1 = open(prediction_file, 'r')
Lines = file1.readlines()
#final_lines = [" ".join(line) for line in Lines]
#spaced_amino_s = [char+' ' for line in Lines for char in line]
#amino_string_s = "".join(spaced_amino_s)

In [ ]:
sequences = []
ctr = 0
test_lines = Lines[0:10]
file_write = open("output_secondary.txt","w")
for line in Lines:
  line_str = line.rstrip("\n")
  line_str = clip_long_seq(line_str, 1024)
  token_ids = torch.tensor([tokenizer.encode(line_str)])
  output = model(token_ids)
  sq = torch.squeeze(output[0],dim=0)
  avg_output = torch.mean(sq,0)
  avg_output_corr = torch.clip(avg_output, min=-1,max=1)
  #print(torch.mean(avg_output_corr))
  ctr += 1
  file_write.write(str(avg_output_corr) + "\n")
  #pooled_output = output[1]
  #file_write.write(str(avg_output_corr) + "\n")

In [ ]:
print(len(final_lines))
print(len(final_lines[1]))
print(len(final_lines[1].rstrip("\n")))

85450
2049
2048
